In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os

import pandas as pd

from pathlib import Path
from datetime import datetime, timedelta
from pytz import timezone
from tqdm.auto import tqdm 

from functional import seq # for chaining filters and maps

# Data processing

```
interim/
    aligned_data/
        $part_id/
            audio/
            empatica/
            edf/
            chillp/
            timeline.csv
            metadata.json
            markers.csv
```



In [4]:
raw_empatica_dir = Path('../../data/raw/uz_study/empatica_data/')
edf_dir = Path('../../data/raw/uz_study/study_data/Physiological Data/Files/')
raw_chillp_dir = Path('../../data/raw/uz_study/chillp_data/')
aligned_data_dir = Path('../../data/interim/uz_study/aligned_data/')

In [5]:
df_mapping = pd.read_csv('../../data/interim/uz_study/aligned_data/mapped_data.csv')

## Empatica data

In [6]:
from context_aware.interfaces.empatica.e4_connect import E4ConnectProcessor
proc = E4ConnectProcessor()

ModuleNotFoundError: No module named 'context_aware'

In [8]:
# Put the data into the correct folder
for _, row in tqdm(df_mapping.dropna(subset=['directory_empatica']).iterrows()):
    proc.process_unzipped_folder(folder_path=f'../../data/raw/uz_study/empatica_data/{row["directory_empatica"]}', 
                                 save_dir=f'../../data/interim/uz_study/aligned_data/{row["Participant"]}/empatica/', 
                                 save_type='feather')

NameError: name 'proc' is not defined

## EDF data

In [9]:
import mne
import neurokit2 as nk


Bad key "text.kerning_factor" on line 4 in
C:\Users\mitch\AppData\Local\Continuum\anaconda3\envs\EEGStudy2020\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [10]:
df_mapping['t_start_edf'] = None

In [12]:
fs = 512 # all signals within the .edfs are sampled at 512Hz

for edf_file in tqdm(sorted(list(edf_dir.glob('*Full*.edf')))):
        # only retain the .edf files of which we have a mapping to the participant ID
        edf_id = int(edf_file.name.split('EEG_')[1][:2])
        if edf_id in df_mapping.edf_marker_id.dropna().values:
            raw= mne.io.read_raw_edf(edf_file, verbose=False)
            row = df_mapping[df_mapping.edf_marker_id == edf_id]
            part_id = row['Participant'].values[0]
            print(edf_file.name)
            print(f'edf: {edf_file.name}\tpart id: {part_id}')
            print(f'edf time:{datetime.fromtimestamp(raw.annotations.orig_time)} \t t_study: {row.t_study.iloc[0]} \t old_marker: {row.old_marker_time.iloc[0]}')

            # Assert that the trigger sequence is the same as in the corresponding eprime file
            df_marker = pd.read_csv(aligned_data_dir.joinpath(str(part_id)).joinpath('marker.csv'))
            low_range, high_range = list(range(1, 68)), list(range(68, 135))
            edf_descr, marker_descr = seq(raw.annotations.description[2:]).map(int), seq(df_marker.Description.values)
            assert edf_descr.filter(lambda x: x in low_range) == marker_descr.filter(lambda x: x in low_range)
            assert edf_descr.filter(lambda x: x in high_range) == marker_descr.filter(lambda x: x in high_range)
            
            # set the correct time in the .edf files
            # dus gewoon in de edf file de correcte date + timezone zeeten en gucci drippin' 
            df_edf = raw.to_data_frame() # 
            fs = 512
            # TODO -> fix code here for common triggers ... 
            # of misschien te experiment end gebruiken voor alignment 
            
#             def get_t_start_trigger(df_marker, raw, trigger):
#                 marker_row = df_marker[df_marker.Description == trigger]
#                 # print(marker_row)
#                 onset = raw.annotations.onset[list(raw.annotations.description).index(str(trigger))]
#                 assert len(marker_row) == 1 
#                 assert int(onset) == marker_row.Position.values[0] // 512.0, print(f"onset: {onset}, row position {row.Position.values[0]//512}")
#                 t_start = pd.to_datetime(marker_row['timestamp'], infer_datetime_format=True).iloc[-1] - timedelta(seconds=onset)
#                 return t_start
#             common_trigger = edf_descr.filter(lambda x: x in low_range)[0]  # define a common trigger whic will be used to set the correct date
#             t_start = get_t_start_trigger(df_marker, raw, trigger=common_trigger)
#                         print('mapped on common trigger   :', t_start)
#             print('mapped on experiment start :', get_t_start_trigger(df_marker, raw, trigger=150))
#             print('mapped on experiment end   :', get_t_start_trigger(df_marker, raw, trigger=151))
#             print('mapped on Stress3 SAM start:', get_t_start_trigger(df_marker, raw, trigger=174))
            t_start = pd.to_datetime(row['t_start_eprime'], infer_datetime_format=True).dt.tz_localize(None).iloc[-1]

            df_edf.index = pd.date_range(start=t_start, tz=timezone('Europe/Brussels'), freq=f'{1e9/ fs}N', periods=len(df_edf))
            df_mapping.at[df_mapping.Participant == part_id, 't_start_edf'] = t_start

        # save the edf dataframe
            save_dir = aligned_data_dir.joinpath(str(part_id)).joinpath('edf')
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            # use neurokit to create an ecg and resp dataframe
            df_ecg = nk.ecg_process(df_edf['ECG1+'].values, sampling_rate=fs)[0].set_index(df_edf.index)
            df_resp = nk.rsp_process(df_edf['R+'].values, sampling_rate=fs)[0].set_index(df_edf.index)
         
            df_edf.reset_index(drop=False).rename(columns={'index': 'timestamp'}).to_feather(save_dir.joinpath('edf.feather'))
            df_ecg.reset_index(drop=False).rename(columns={'index': 'timestamp'}).to_feather(save_dir.joinpath('ecg.feather'))
            df_resp.reset_index(drop=False).rename(columns={'index': 'timestamp'}).to_feather(save_dir.joinpath('resp.feather'))

df_mapping.to_csv('../../data/interim/uz_study/aligned_data/mapped_data.csv', index=False)

EEG_06 - Full (Raw).edf
edf: EEG_06 - Full (Raw).edf	part id: 5
edf time:2020-01-01 10:43:08 	 t_study: 2020-01-28 09:48:11.673000+01:00 	 old_marker: 2020-01-28 09:00:46.701172
Converting "time" to "<class 'numpy.int64'>"...



KeyboardInterrupt: 

---

## Chill+ data

In [14]:
chillp_raw_dir = Path('../../../data/raw/uz_study/chillp_data/')

In [26]:
for _, row in df_mapping.iterrows():
    part_id = row['Participant']
    chillp_data_dir = chillp_raw_dir.joinpath(str(part_id))
    
    if not aligned_data_dir.joinpath(str(part_id)).is_dir():
        print(f'skipping {part_id}')
        continue
    
    if chillp_data_dir.is_dir():
        chillp_aligned_dir = aligned_data_dir.joinpath(str(part_id)).joinpath('chillp')
        if not chillp_aligned_dir.is_dir():
            os.makedirs(chillp_aligned_dir)
        # iterate over all the feathers
        for feather_file in chillp_data_dir.glob('*.feather'):
            !cp -pr {str(feather_file)} {str(aligned_data_dir.joinpath(str(part_id)).joinpath('chillp')) + '/'}

skipping 1
skipping 61


---

## Speech data

In [21]:
audio_data_dir = Path('../../../data/raw/uz_study/study_data/Tablet\ Data/Audio/')
aligned_data_dir = Path('../../../data/interim/uz_study/aligned_data/')

In [21]:
tablet_data_mapping = pd.read_csv('../../../data/raw/uz_study/study_data/Tablet Data/Behavioural/results_09_03_2020.csv')

> TODO: I assume that the id corresponds with participant id

In [22]:
# discard the invalid id's
tablet_data_mapping = tablet_data_mapping[tablet_data_mapping['id'].map(int) < 99]

In [23]:
# perfect -> participant 25 & 45 hebben onafgewerkte data én zitten niet in de dataframe
for _, row in tablet_data_mapping.iterrows():
    part_id = row['id']
    save_dir = aligned_data_dir.joinpath(str(part_id)).joinpath('audio/')
    if not save_dir.is_dir():
        os.makedirs(save_dir)
    !cp -pr {str(audio_data_dir.joinpath(str(part_id))) + '/*'} {save_dir} 

---

## Questionnaires

TODO:
 * kijken voor valence arousal te mappen
 * kijken voor nog 